In [13]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

#### Retorna a categoria do local

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    return categories_list[0]['name']

#### Informações para acessar a API do foursquare

In [27]:
CLIENT_ID = '55LWIFZLZNZPSTAZZQKJXBD11BSBCUZTZPIXWVV4RFZJP1HS' # your Foursquare ID
CLIENT_SECRET = 'RPCOWGHQY5YNQUCHQUTC5VLKEYFGZN15FSIYTK5GNXNZ301E' # your Foursquare Secret
VERSION = '20180604'

#### Informações da cidade delimitada

In [69]:
latitude_nyork = 40.730610
longitude_nyork = -73.935242
radius = 5000
LIMIT = 1000
QUERY = 'beer'

#### Coletando as informações dos locais

In [70]:
url = 'https://api.foursquare.com/v2/venues/search?intent=checkin&query={}&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    QUERY, CLIENT_ID, CLIENT_SECRET, latitude_nyork, longitude_nyork, VERSION, radius, LIMIT)
            
# make the GET request
results = requests.get(url).json()

In [72]:
venues = results['response']['venues'] 
nearby_venues = json_normalize(venues)
filtered_columns = ['id', 'name', 'categories', 'location.lat', 'location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.tail(10)

,id,name,categories,lat,lng
40,55b88ad7498e1653a9ffaecb,"Joshua D. Beer, MD",Doctor's Office,40.765456,-73.980542
41,4fc92684e4b0a40e5e28ca2c,Beer Cart LIRR Hunterspoint Ave,Bar,40.742457,-73.946473
42,5079cb44e4b08ed647bf5587,Beer tour,Liquor Store,40.721420,-73.957562
43,571abec5498e9a18b35e43d4,Bowery Beer Garden,Beer Garden,40.717192,-73.995312
44,51fc6b8d498e903258d2f237,Antler Beer and Wine Dispensary,Bar,40.719695,-73.990251
45,565cbec4498e7ec1b08e8182,Norwinds Beer Garden,Beer Garden,40.704114,-73.930618
46,4b37ce87f964a520884625e3,Duane Reade,Pharmacy,40.719128,-73.963260
47,4bb6b10446d4a59322b0c6c0,Beer Sandwiches,None,40.711003,-73.949211
48,5155b97de4b0d9a3f4e12a02,Union Beer Graphics Department,Office,40.715082,-73.928637
49,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476


In [93]:
users_total = pd.DataFrame()
for venue in nearby_venues.iterrows():
    url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}&limit={}'.format(
    venue[1]['id'], CLIENT_ID, CLIENT_SECRET, VERSION, 2000)
    results = requests.get(url).json()
    if (results['response']['likes']['count'] > 0) & ('items' in results['response']['likes']):
        results_2 = results['response']['likes']['items']
        users = json_normalize(results_2)[['id', 'firstName', 'gender']]
        users['avenue_id'] = venue[1]['id']
        users['avenue_name'] = venue[1]['name']
        users['avenue_categories'] = venue[1]['categories']
        users['avenue_lat'] = venue[1]['lat']
        users['avenue_lng'] = venue[1]['lng']
        if users_total.empty:
            users_total = users
        else:
            users_total = pd.concat([users_total, users], ignore_index=True)

In [94]:
users_total.tail()

,id,firstName,gender,avenue_id,avenue_name,avenue_categories,avenue_lat,avenue_lng
9257,39991457,Jannis,female,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9258,6909884,Carol 🎄⛄,female,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9259,3088472,Chase,male,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9260,14184080,John,male,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476
9261,2631756,Tokuyuki,male,4ced1fd813aea143d326659f,Tavern on the Green,American Restaurant,40.772256,-73.977476


In [95]:
users_total.to_csv('curtidas_cervejarias.csv', index=False)